# Task 1.2: English Syntax Generalization Practice
We used the English GUM corpus with 233k entries maintained by Georgetown University Linguistics students. The text comes from multiple media formats. Our goal was to examine a generalization on verbs and their direct object pronouns within the corpus. The generalization accurately described the relevant sentences in the corpus.

## Our Generalization
**Expected Generalization:** Pronoun direct objects immediately follow their linked verbs.

An example that illustrates this generalization is provided below.

“First, our experimental subjects lived in a large enclosure under conditions that **allowed them** to exercise all day long.”

## Results
We used a function called 'finderskeepers' to search the corpus for sentences that contain pronoun (direct) objects linked to verbs. If those pronouns immediately followed the verb, then they fit our generalization. Otherwise, they were flagged as exceptions.

About 86% of the corpus fit our generalization; direct object pronouns indeed follow their linked verbs. The exceptions to this are when adverbs modify the verb, adjectives modify the pronoun, wh-questions, a passivized verb requires argument movement to bring the direct object pronoun *forward* in the sentence, among others. Examples are provided below.

## Examples of Exceptions
### Wh-Question
"If your professor comes into an early morning class holding a mug of liquid, **what** do you assume she is **drinking**?"

### Stammer
“You know, **documenting**, uh, uh, **whatever**.”

### Modifying Preposition
"And I said ‘Ben, **pick** me out **something**, you've got fifty bucks to spend.'"

### Dialogue
"Whitmore told LA Weekly that on October 6 after traveling back from Florida, Montalvo ‘walked into lobby of the East L.A. station and turned himself in’, and **told** the police, ‘**everything** he did’.”

### Wh-Movement
“Malaysia and Indonesia have maintained a policy of turning away boats of migrants **which**, according to AFP, the United Nations and United States have both **criticised**.”

### Movement
"**This** I would have **asked** him had he not been so far away, but he was very far, and could not be seen at all when he drew nigh that gigantic reef."

In [ ]:
# english code

# Tasks 1.1 & 1.3: Swedish Verb Negation Generalization
We used the Swedish LinES corpus (from the Parallel Treebank of the same name) that includes just over 100k Swedish translations from English text. Our goal was to examine a generalization on Swedish verb negation. Our results conflicted with our expectations, and so we performed further examination to search for a different possible generalization.

## Our Generalization
**Expectated Generalization:** Negation words immediately follow the verb they negate.

The below example illustrates this generalization. 

“Hon **svarade inte**.”
(She didn't answer.)
('inte' = 'not', linked words are bolded)

## Results & Discussion
We used a function called 'swedishchecker' to search the corpus for negation words linked to verbs. Then we filtered instances where the negation came immediately after the verb; those examples fit our generalization. The other sentences were cached as exceptions.

Only about 21% of our corpus fit the expected generalization. There were a lot of exceptions ranging from embedded sentences, questions, auxiliaries, verb-object switches, and the list goes on. Some examples of these are provided below:


## Secondary Corpus Test
When tested with a slightly smaller corpus (96k entries) called Talbanken from Lund University. The sentences were taken from various text genres like textbooks, brochures, and newspaper articles. We found similar results to the above, where 22% of the sentences with negated verbs actually fit our generalization. This leads us to believe that the translation bias in our first corpus may not be the reason that our generalization fits so poorly.

In [ ]:
# swedish code

# Generalization Exceptions
Our generalization seems to hold on simple sentences with little to no nuance.

 Examples include:

“Hon svarade inte.” -“She didn't answer.”

"Hon talar inte jiddisch?" - “She doesn't speak Yiddish?”

These sentences relay straightforward information and do not contain many flourishes in speech. If we were to only consider such sentencs, our generalization holds with 21% accuracy on verb negations.

However, if we take into account more detailed sentences, we see a diffrerent result. In examining our initial results, there were two main exceptions we identified that change the location of negation. If we include this nuance, accuracy increases to 44%

- **Auxiliary verbs:** if auxilary verbs are present, the negation is placed between the auxiliary and the main (head) verb

“Hans sekreterare hade inte ringt det samtal hon hade fått instruktioner om.”

“His secretary had not made the instructed call.”
	
- **Embedded clauses:** if there is an embedded clause, the negation follows the subject of the clause and preceedes the main (head) verb. This seems to happen because of rules regarding VPs in Swedish

“Jag har suttit här tålmodigt och jag finner det anmärkningsvärt att ni inte ropar upp mig.”

“I have sat here patiently and I find it quite extraordinary that you are not calling me.”

While 44% accuracy might seem low, this result can be explained by recognizing a feature of Swedish that lets words be reordered to put emphasis on certain aspects of the sentence. For example, Object Shift allows for the object of a verb to swap places with the negation, while still producing a gramatical sentence

“Jag förstår det inte alls.”

“I do not understand it at all.”


In [ ]:
# any additional code

# Task 2

In [ ]:
# verb frequency method

The verbs chosen are randomly sampled from the top 20% of verbs when sorted by frequency and the next 20% of most frequently used verbs -- five from each category. Frequency was determined by tallying the lemmas of each verb.

In [ ]:
# gen_sets method

For any verb's *lemma* in the set of Swedish sentences, this method generates a dictionary containing sets of each subjects, objects, modifiers, preceding words, and following words corresponding to the given verb. The set of modifiers for the verb only contains adverbs, (or those with the dependency relationship "advmod" to the verb), but not other modifiers like negation, prepositions, or auxiliaries. Including would have likely skewed our results by adding more noise, since semantically, adverbs might be more significant. However, it may have been beneficial to have include other modifiers as well.

In [ ]:
# ???

After we have the sets of words that we need, we need to make the Word2Vec model. This is what the following function does, returning it in the form of a space so that we can use gensim library functions on it:

In [3]:
# word 2 vec model stuff